In [1]:
import os

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

22/03/30 07:45:10 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/30 07:45:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6b48126-b3f7-4806-9354-66ff1b2c0df9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 539ms :: artifacts dl 10ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group-sg11")\
  .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() 

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [8]:
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [9]:
stockTickDf = jsonDf.select (F.col("value.*"))
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [10]:
stockTickDf = stockTickDf\
                .withColumn("timestampTemp", (F.col("timestamp") / 1000).cast("timestamp"))\
                .withColumn("trade_time", F.date_trunc("minute", F.col("timestampTemp")))\
                .drop("timestamp")\
                .drop("timestampTemp")\
                .withColumnRenamed("trade_time", "timestamp")

In [11]:
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
(stockTickDf.withColumn("year", F.date_format(F.col("timestamp"), "yyyy"))
            .withColumn("month", F.date_format(F.col("timestamp"), "MM"))
            .withColumn("day", F.date_format(F.col("timestamp"), "dd"))
            .withColumn("hour",F.date_format(F.col("timestamp"), "HH"))
            .withColumn("SYMBOL", F.col("symbol"))
            .writeStream
            .trigger(processingTime ='65 seconds')\
            .queryName("write to csv")
            .format("csv")
            .option("path", "hdfs://localhost:9000/dump-1min")
            .option("header", True)
            .option("checkpointLocation", "hdfs://localhost:9000/checkpoint/tickscsvtohdfs6")
            .partitionBy("year", "month", "day", "hour", "SYMBOL")
            .option("truncate", False)
            .start()
)

22/03/28 04:01:13 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
java.lang.IllegalStateException: Cannot fetch offset 1451 (GroupId: spark-kafka-source-5c7574d7-0e4b-4c4d-8cb2-7acd8b2a88b0--171825179-executor, TopicPartition: stock-ticks-0). 
Some data may have been lost because they are not available in Kafka any more; either the
 data was aged out by Kafka or the topic may have been deleted before all the data in the
 topic was processed. If you don't want your streaming query to fail on such cases, set the
 source option "failOnDataLoss" to "false".
    
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer$.org$apache$spark$sql$kafka010$consumer$KafkaDataConsumer$$reportDataLoss0(KafkaDataConsumer.scala:664)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.reportDataLoss(KafkaDataConsumer.scala:599)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:331)
	at org.apache.spark.util.UninterruptibleThre